<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/utilities/examples_save_Load_with_safetensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Safetensors can be used to save and load tensors in a safe and performant way.
What do we mean by safe ?
With safe we mean Safetensor doesn't allow the execution of arbitrary code while loading of data.
But Safetensors has more benefits just like that. The library supports lazy loading, so we can inspects which tensors are in a file without loading the whole file and and just load required tensors.
Zero-copy: safetensor doesn't requite more memory when reading then the original file.

First let us create two tensors as a dictionary

In [ ]:
import torch
from safetensors.torch import save_file, safe_open

tensors = {
    "embedding": torch.zeros((2, 2)),
    "attention": torch.zeros((2, 3))
}


Now we can use the safetensor function save_file to save the dictionary to a file.

In [ ]:
save_file(tensors, "model.safetensors")

Reading in the file is simple as well



In [ ]:
tensors = {}
with safe_open("model.safetensors", framework="pt") as f:    # , device=0 as parameter when we
    for k in f.keys():
        tensors[k] = f.get_tensor(k) # loads the full tensor given a key
print(tensors)

{'attention': tensor([[0., 0., 0.],
        [0., 0., 0.]]), 'embedding': tensor([[0., 0.],
        [0., 0.]])}


if we want to load the tensor directly into GPU we can add a "device" parameter

with safe_open("model.safetensors", framework="pt", device=0) as f:

SafeTensors also supports lazy loading and loading of parts of the files. This is helpful for large file for example to avoid loading meta data.

We can also just load a slice of a tensor

In [ ]:
tensors = {}
with safe_open("model.safetensors", framework="pt") as f:
    tensor_slice = f.get_slice("embedding")
    vocab_size, hidden_dim = tensor_slice.get_shape()
    tensor = tensor_slice[:, :hidden_dim-1]
print (tensor)

tensor([[0.],
        [0.]])


loading of model state is also directly supported by the library. Instead of calling model.state_dict() and save the dictionary we can simply pass in the model to the save_model() model. for loading the state dictionary we can use the function load_model()

In [ ]:
from safetensors.torch import load_model, save_model

save_model(model, "model.safetensors")
# Instead of save_file(model.state_dict(), "model.safetensors")

load_model(model, "model.safetensors")
# Instead of model.load_state_dict(load_file("model.safetensors"))

NameError: name 'model' is not defined